In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import date, timedelta
import matplotlib.pyplot as plt
import os
import re
from openpyxl import load_workbook


from Fase3_Aux_Func import (agrupamento_periodico,
                            ajuste_datas_historica,
                            conversao_array,
                            plot_serie,
                            plot_decomposicao,
                            suavizacao_simples,
                            suavizacao_exponencial,
                            holt_winters,
                            arma,
                            arima_sazonal,
                            analise_residuos,
                            check_stationarity,
                            grid_search_best_model,
                            load_models,
                            criar_forecast)

In [8]:
global local_origem 
global data_corte_treinamento

local_origem = "C:/Lab Transp/Dados Processados/"
local_destino = "C:/Lab Transp/Dados Processados/"
data_corte_treinamento = '2024-06-30'

### 1. Carregamento dos Dados e Arquivos dos Modelos

In [4]:
'''
LEMBRA DE MUDAR O NOME DO ARQUIVO PARA df_final_tratado DEPOIS DE AUTALIZAR O BD_BILHETAGEM E RODAR O DF TRATADO
'''
df_bilhetagem = pd.read_csv(local_origem + "/df_final_agrupado_ano_mes.csv", sep=",", decimal="," )
df_bilhetagem['Volume_Passageiros_BU_VT_DIN'] = df_bilhetagem['Volume_Passageiros_BU_VT_DIN'].astype(int)
df_bilhetagem.head()

,Ano,Mes,Cidade,Zona,Sub_Prefeitura,Volume_Passageiros_BU_VT_DIN
0,2014,1,TOTAL,CENTRO,SE,19599200
1,2014,1,TOTAL,LESTE,ARICANDUVA-FORMOSA-CARRAO,1234500
2,2014,1,TOTAL,LESTE,ERMELINO MATARAZZO,37373
3,2014,1,TOTAL,LESTE,GUAIANASES,2652418
4,2014,1,TOTAL,LESTE,ITAIM PAULISTA,623072


In [5]:
'''
Irá realizar os ajustes de datas necessários, periorizando por dia, além de quebrar em 2 arquivos
'''
df_subpref, df_zonas, df_total = agrupamento_periodico(df_bilhetagem)

In [6]:
# Dicionário com os caminhos dos arquivos pickle dos modelos
model_file_subpref = 'best_models_subpref_{0}.pkl'
model_file_zona = 'best_models_zona_{0}.pkl'
model_file_total = 'best_models_total.pkl'

### 2. Criação dos Forecasts

In [ ]:
# Loop para fazer o forecast e salvar os resultados para cada subprefeitura e zona
for sub_prefeitura in df_subpref['Sub_Prefeitura'].unique():
    output_file = f"{local_origem}/Forecasts/forecast_subpref_{sub_prefeitura}.csv"
    
    criar_forecast(df_subpref, 
                   'Sub_Prefeitura',
                   sub_prefeitura, 
                   model_file_subpref,
                   output_file,
                   12)

print("Forecasts subprefeituras realizados e salvos com sucesso.")

In [ ]:
for zona in df_zonas['Zona'].unique():
    output_file = f"{local_origem}/Forecasts/forecast_zona_{zona}.csv"
    criar_forecast(df_zonas, 'Zona', zona, model_file_zona, output_file, 12)
    
print("Forecasts Zonas realizados e salvos com sucesso.")

In [12]:
#Total
output_file = f"{local_origem}/Forecasts/forecast_TOTAL.csv"
criar_forecast(df_total, 'Cidade', 'TOTAL', model_file_total, output_file, 12)
print("Forecasts para Total realizados e salvos com sucesso.")

Processando Cidade = TOTAL
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em C:/Users/f112596/Desktop/Projetos/DO/Lab Transp/Dados Processados/Forecasts/forecast_TOTAL.csv
Forecasts para Total realizados e salvos com sucesso.


### 3. Consolidação dos Arquivos

In [13]:
arquivo_saida = f'{local_origem}/Forecasts/Forecast_Linhas.xlsx'
arquivo_saida

'C:/Users/f112596/Desktop/Projetos/DO/Lab Transp/Dados Processados/Forecasts/Forecast_Linhas.xlsx'

In [20]:
for arquivo_forecast in os.listdir(f'{local_origem}/Forecasts'):
    if arquivo_forecast.endswith('TOTAL.csv'):
        print(arquivo_forecast)
        sheet_name_match = re.search(r'^[^_]*_[^_]*_([^.]*)', 
                                     arquivo_forecast)
        print(sheet_name_match)

forecast_TOTAL.csv
None


In [21]:
#Consolidando os CSVs 
with pd.ExcelWriter(arquivo_saida, engine='openpyxl') as excel_writer:
    for arquivo_forecast in os.listdir(f'{local_origem}/Forecasts'):
        if arquivo_forecast.endswith('.csv'):
            
            df = pd.read_csv(os.path.join(f'{local_origem}/Forecasts', arquivo_forecast),
                             sep=";", decimal=",", encoding='latin-1')
            
            sheet_name_match = re.search(r'^[^_]*_[^_]*_([^.]*)', arquivo_forecast)

            if sheet_name_match :
                sheet_name = sheet_name_match.group(1)
                
            elif arquivo_forecast=='forecast_TOTAL.csv':
                sheet_name = 'TOTAL'
                
            df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

#Auto Ajuste nas Collunas
wb = load_workbook(arquivo_saida)

for sheet in wb.sheetnames:
    ws = wb[sheet]
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter
        for cell in col:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[col_letter].width = max_length + 2  # Ajuste para melhor visualização

wb.save(arquivo_saida)

print("Os arquivos CSV foram consolidados!")

Os arquivos CSV foram consolidados!
